# FEgen V1.1

When entering parameter values using scientific notation, use **e** notation. (E.g. for 12000 enter 1.2e4, for -0.000012 enter -1.2e-5)
<table>
    <thead >
        <tr>
          <th> Options </th>
          <th>Details </th> 
        </tr>
    </thead>
    <tbody>
        <tr>
          <th>One Emitter</th>
          <td>Select for single emitter</td>
        </tr>
        <tr>
          <th>Emitter Grid</th>
          <td>Select to create a square grid of emitters. See parameters for Size <br> of Grid and Grid Spacing for details on specifying grid size.</td>
        </tr>
        <tr>
          <th>Custom Emission Pattern</th>
          <td> Select to create a custom pattern of emitters. See parameter for <br>Locations of Emitters for details on specifying emitter locations. </td>
        </tr>
        <tr>
            <th>Charge per RF Cycle</th>
            <td>Select for charge in single RF Cycle</td>
        </tr>
        <tr>
            <th>Charge per Pulse Length</th>
            <td>Select for charge over multiple RF Cycles. Pulse Length<br> and frequency are used to calculate charge per cycle $$q=\frac{q_{total}}{\omega * PL}$$</td>
            <th></th>
        </tr>
        <tr>
            <th>Charge per Emitter Radius</th>
            <td>Select to scale the total charge to the emitter radius</td>
        </tr>  
  </tbody>
</table>

----------

<table>
    <thead >
        <tr>
          <th>Parameters </th>
          <th>Details </th>
          <th>Units</th>  
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>Total Charge</th>
            <td>Charge per emitter (e.g. for 1 nC charge over 10 emitters, one <br>would enter 0.1 nC)</td>
            <th>nC</th>
        </tr>
        <tr>
            <th>Initial Energy</th>
            <td>Initial energy distribution of the cathode surface (default=0.1 eV)</td>
            <th>eV</th>
        </tr>
        <tr>
            <th>Local Field</th>
            <td>Equal to $\beta$ times applied field where $\beta$ is the field <br>enhancement factor</td>
            <th>GV/m</th>
        </tr>
        <tr>
            <th>Phase Shift</th>
            <td>Accounts for phase shift of the field. Default value is $180^\circ$ <br> due to a $90^\circ$ phase shift from the cosine and sine expansions <br> (used in Superfish and ASTRA/GPT, respectively) and an <br> additional $90^\circ$ shift such that the reference particle is at the <br>peak field at the time of emission. </td>
            <th>$^\circ$ (deg)</th>
        </tr>
        <tr>
            <th>RF Phase</th>
            <td>Phase window of the RF cycle. Default of emitting over full $360^\circ$, <br> phase window can be specified upon entry.</td>
            <th>$^\circ$ (deg)</th>
        </tr>
        <tr>
            <th>Frequency</th>
            <td>Default option provided is for the Argonne Cathode<br>Teststand (ACT) for which FEgen was originally intended where<br>the default frequency is the L-band operational frequency <br>of the ACT. Alternate frequencies can be entered by user. </td>
            <th>GHz</th>
        </tr>
        <tr>
            <th>Size of Grid</th>
            <td>The length of one side of a square grid of emitters<br> (e.g. Size of Grid = 7 will create a 7x7 grid of 49 total emitters)</td>
            <th></th>
        </tr>
        <tr>
            <th>Grid Spacing</th>
            <td>The distance between emitters when grid of emitters is selected</td>
            <th>mm</th>
        </tr>
        <tr>
            <th>Locations of Emitters</th>
            <td>The location of each emitter in meters. See note at the top <br> of the notebook on how to enter values using scientific <br> notation. Ensure that the number of emitters entered coincides <br> with the number of x,y pairs given. To enter locations, separate <br> respective x and y with a comma and place a semicolon between <br> x,y pairs. Do not place a semicolon after the last x,y pair. <br> Straying from this format will result in an error</td>
            <th>mm</th>
        </tr>
        <tr>
            <th>Current</th>
            <td>For the pulse power option, current and pulse length <br> are used to calculate charge $$q=I*PL$$</td>
            <th>amps</th>
        </tr>
        
  </tbody>
</table>



In [4]:
import tkinter as tk
from tkinter import filedialog
import numpy as np
from scipy.optimize import curve_fit
from scipy import stats
from tkinter.ttk import *
from decimal import *
getcontext().prec = 30
from mpmath import mp


class FEgen_v1():
    """
    This Python GUI is designed to replicate and simplify the function of ASTRA (A Space Charge Tracking 
    Algorithm) to create initial particle distribution files that can be loaded into GPT (General Particle Tracer). 
    Based on user inputs, this program creates initial particle distributions for position, momentum, time, and
    total charge, in addition to having the species (1) and flags (-1) stored in the generated file. This program
    is specific to replicating ASTRA's isotropic momentum distribution with a uniform distribution of energy.
    
    This version is limited to a planar geometry with options for both RF Cycle and Pulse Power capabilities.

    To use the GUI, one needs to install Python3 with the Tkinter, Numpy, Scipy, Decimal, and mpmath packages.
    
    Cite as:  E. Jevarjian, M. Schneider, S.V. Baryshev. MSU-MAM-001 (2020).
    jevarji1@msu.edu
    
    
    """
    def __init__(self):
        # Setting up GUI Window
        self.window = tk.Tk()
        self.window.title("FEgen")
        self.window.configure(bg='gray')

        # Setting up frames within main window
        content=tk.Frame(self.window,bg='gray')
        emitter_frame = tk.Frame(content,relief='sunken',borderwidth=5,height=200,width=200)
        all_charge_frames = tk.Frame(content,relief='sunken',borderwidth=5,height=200,width=100)
        charge_entry_frame = tk.Frame(all_charge_frames,relief='sunken',borderwidth=5,height=200,width=100)
        charge_per_pulsePL_frame = tk.Frame(all_charge_frames,relief='sunken',borderwidth=5,height=200,width=100)
        self.charge_grid_custom_frame = tk.Frame(all_charge_frames,relief='sunken',borderwidth=5,height=200,width=100)
        input_frame = tk.Frame(content,relief='sunken',borderwidth=5,height=100,width=100)
        self.if_grid_frame = tk.Frame(content,relief='sunken',borderwidth=5,height=100,width=100)
        self.if_custom_frame = tk.Frame(content,relief='sunken',borderwidth=5,height=100,width=100)
        save_frame = tk.Frame(content,relief='sunken',borderwidth=5,height=100,width=100,bg='#EA7D7D' )
        pulsepower_rf_frame = tk.Frame(content,relief='sunken',borderwidth=5,height=100,width=100)

        # Select RF Cycle or Pulse Power Frame
        self.pulsepower_rf_var = tk.IntVar()
        self.pulsepower_rf_var.set(1)
        rf_option = tk.Radiobutton(pulsepower_rf_frame,text='RF Cycle',variable=self.pulsepower_rf_var,value=1,command=self.pulsepower_rf_command)
        pulsepower_option = tk.Radiobutton(pulsepower_rf_frame,text='Pulse Power',variable=self.pulsepower_rf_var,value=2,command=self.pulsepower_rf_command)
        
        # Emitter Type Frame
        emitter_label = tk.Label(emitter_frame,text = 'Emitter',font="Verdana 15 underline")
        self.emitter_var = tk.IntVar()
        one_emitter = tk.Radiobutton(emitter_frame,text="One emitter",value=1,
                                          variable=self.emitter_var,command=self.change_emitter)
        grid_emitter = tk.Radiobutton(emitter_frame,text='Grid',value=2,
                                           variable=self.emitter_var,command=self.change_emitter)
        custom_emitter = tk.Radiobutton(emitter_frame,text='Custom Emission Pattern',value=3,
                                            variable=self.emitter_var,command=self.change_emitter)
        # Charge Entry Frame
        self.charge_text = tk.Label(charge_entry_frame,text='Total Charge [nC]:')
        self.charge_var = tk.Entry(charge_entry_frame,width=10)

        # Charge per Pulse Length or per RF Cycle Frame
        self.charge_pulse_type_var = tk.IntVar()
        self.per_PL = tk.Radiobutton(charge_per_pulsePL_frame,text='Charge per pulse length',value=1,
                               variable = self.charge_pulse_type_var)
        self.per_pulse = tk.Radiobutton(charge_per_pulsePL_frame,text='Charge per RF Cycle',value=2,
                                  variable=self.charge_pulse_type_var)
        PL_text = tk.Label(charge_per_pulsePL_frame, text='Pulse length[s]:')
        self.PL_entry = tk.Entry(charge_per_pulsePL_frame, width=8)
        # Charge per Cathode or Emitter Radius Frame (Options for Grid of Emitters or Custom Pattern)
        self.charge_rad_type_var = tk.IntVar()
        per_cath_rad = tk.Radiobutton(self.charge_grid_custom_frame,text='Charge per cathode radius',value=1,variable=self.charge_rad_type_var)
        per_emit_rad = tk.Radiobutton(self.charge_grid_custom_frame,text='Charge per emitter radius',value=2,variable=self.charge_rad_type_var)
        emit_rad_text = tk.Label(self.charge_grid_custom_frame,text='     Emitter radius [m]:')
        self.emit_rad_entry = tk.Entry(self.charge_grid_custom_frame,width=8)

        # Entries for if Grid of Emitters is selected
        grid_rad_text = tk.Label(self.if_grid_frame,text='Size of Grid:\n(ex: 7=7x7 grid)')
        self.grid_rad_entry = tk.Entry(self.if_grid_frame,width = 4)
        grid_space_text = tk.Label(self.if_grid_frame,text='Grid Spacing [mm]:')
        self.grid_space_entry = tk.Entry(self.if_grid_frame,width=5)

        # Entries for if Custom Pattern of Emitters is selected
        num_emit_text = tk.Label(self.if_custom_frame,text='Total number of emitters:')
        self.num_emit_entry = tk.Entry(self.if_custom_frame,width=4)
        emit_loc_text = tk.Label(self.if_custom_frame,text='Enter locations of emitters [mm]:')
        self.emit_loc_entry = tk.Text(self.if_custom_frame,height=3,width=28,bd=5,bg='#D5D0CF')
        self.emit_loc_entry.insert("1.0",'x1,y1;x2,y2;...')

        # Input Paramters
        inputs_title = tk.Label(input_frame,text='Inputs',font="Verdana 15 underline")
        empty1 = tk.Label(input_frame,text='')
        self.phi_text = tk.Label(input_frame,text='Workfunction [eV]:')
        self.phi_entry = tk.Entry(input_frame,width=10)
        self.cath_rad_var = tk.Entry(input_frame,width=10)
        cath_rad_text = tk.Label(input_frame,text='Cathode Radius [m]:')
        n_particles_text = tk.Label(input_frame,text="Number of particles:")
        self.n_particles_entry = tk.Entry(input_frame,width=10)
        Ek_text = tk.Label(input_frame,text='Initial Energy [eV]:')
        self.Ek_entry = tk.Entry(input_frame,width=10)
        self.Ek_entry.insert(0,'0.1')
        self.local_field_text = tk.Label(input_frame,text="Local Field [GV/m]:")
        self.local_field_entry = tk.Entry(input_frame,width=10)
        self.phase_shift_text = tk.Label(input_frame,text='Phase Shift [deg]:')
        self.phase_shift_entry = tk.Entry(input_frame,width=4)
        self.phase_shift_entry.insert(0,'180')
        self.rf_phase_text = tk.Label(input_frame,text='RF Phase [deg]:')
        self.rf_phase_start = tk.Entry(input_frame,width=3)
        self.rf_phase_start.insert(0,'0')
        self.rf_phase_dash = tk.Label(input_frame,text='to')
        self.rf_phase_end = tk.Entry(input_frame,width=3)
        self.rf_phase_end.insert(0,'359')
        self.freq_text = tk.Label(input_frame,text='Frequency [GHz]:')
        self.act_freq_var = tk.BooleanVar()
        self.act_freq = tk.Radiobutton(input_frame,text='ACT=1.316127688539',variable=self.act_freq_var,value=True)
        self.other_freq = tk.Radiobutton(input_frame,text='Other', variable=self.act_freq_var,value=False)
        self.other_freq_entry = tk.Entry(input_frame,width=15)

        # Frame for saving file
        slct_folder_btn = tk.Button(save_frame,text='Select folder to save file',highlightbackground='#EA7D7D',command=self.slct_folder)
        run_save_btn = tk.Button(save_frame,text='Create file and save',highlightbackground='#EA7D7D',command=self.run)
        empty = tk.Label(save_frame,text='         ',bg='#EA7D7D')
        filename_text = tk.Label(save_frame,text='Enter filename to save as (without extension):',bg='#EA7D7D')
        self.filename_entry = tk.Entry(save_frame,width=35,highlightbackground='#EA7D7D')
        self.progress_var = tk.IntVar()
        self.progress_var.set(0)
        self.progress_bar = Progressbar(save_frame,orient='horizontal', length=100,mode='determinate',variable=self.progress_var)

        # Organizing Frames in Grid Layout
        content.grid(row=0,column=0,columnspan=6,rowspan=13)
        pulsepower_rf_frame.grid(row=0,column=0,rowspan=2,sticky='nwe',columnspan=2)
        emitter_frame.grid(row=2,column=0,rowspan=4,sticky='nwe',columnspan=2)
        all_charge_frames.grid(row=6,column=0,rowspan=6,columnspan=2,sticky='new')
        charge_entry_frame.grid(row=6,column=0,columnspan=2,sticky='ew')
        charge_per_pulsePL_frame.grid(row=7,column=0,columnspan=2,rowspan=3,sticky='ew')
        self.charge_grid_custom_frame.grid(row=10,column=0,columnspan=2,rowspan=3,sticky='ew')
        input_frame.grid(row=0,column=2,rowspan=10,columnspan=4,sticky='new')
        self.if_grid_frame.grid(row=13,column=0,columnspan=2,rowspan=2,sticky='new')
        self.if_custom_frame.grid(row=13,column=0,rowspan=4,columnspan=2,sticky='ew')
        save_frame.grid(row=10,column=2,columnspan=4,rowspan=3,sticky='new')
        
        # Organizing inputs/options in grid layout
        # RF Cycle/Pulse Power options
        rf_option.grid(row=0,column=0,columnspan=2,sticky='nw')
        pulsepower_option.grid(row=1,column=0,columnspan=2,sticky='nw')
        # Emitter type options
        emitter_label.grid(row=2,column=0,columnspan=2)
        one_emitter.grid(row=3,column=0,sticky='w',columnspan=2)
        grid_emitter.grid(row=4,column=0,sticky='w',columnspan=2)
        custom_emitter.grid(row=5,column=0,sticky='w',columnspan=2)
        # Charge Input
        self.charge_text.grid(row=6,column=0,sticky='e')
        self.charge_var.grid(row=6,column=1,sticky='w')
        # Charge per RF cycle or per pulse length option
        self.per_pulse.grid(row=7,column=0,sticky='w',columnspan=2)
        self.per_PL.grid(row=8,column=0,sticky='w',columnspan=2)
        PL_text.grid(row=9,column=0,sticky='e')
        self.PL_entry.grid(row=9,column=1, sticky='e')
        # Charge per emitter radius or cathode radius option 
        per_emit_rad.grid(row=10,column=0,columnspan=2,sticky='w')
        per_cath_rad.grid(row=11,column=0,columnspan=2,sticky='w')
        emit_rad_text.grid(row=12,column=0,sticky='e')
        self.emit_rad_entry.grid(row=12,column=1,sticky='e')
        # Input Parameters
        inputs_title.grid(row=0,column=2,columnspan=4,sticky='s')
        self.phi_text.grid(row=2,column=2,columnspan=2,sticky='e')
        self.phi_entry.grid(row=2,column=4,columnspan=2,sticky='w')
        cath_rad_text.grid(row=3,column=2,columnspan=2,sticky='e')
        self.cath_rad_var.grid(row=3,column=4,columnspan=2,sticky='w')
        n_particles_text.grid(row=4,column=2,columnspan=2,sticky='e')
        self.n_particles_entry.grid(row=4,column=4,columnspan=2,sticky='w')
        Ek_text.grid(row=5,column=2,columnspan=2,sticky='e')
        self.Ek_entry.grid(row=5,column=4,columnspan=2,sticky='w')
        self.local_field_text.grid(row=6,column=2,columnspan=2,sticky='e')
        self.local_field_entry.grid(row=6,column=4,columnspan=2,sticky='w')
        self.phase_shift_text.grid(row=7,column=2,columnspan=2,sticky='e')
        self.phase_shift_entry.grid(row=7,column=4,columnspan=2,sticky='w')
        self.rf_phase_text.grid(row=8,column=2,sticky='e')
        self.rf_phase_start.grid(row=8,column=3,sticky='e')
        self.rf_phase_dash.grid(row=8,column=4)
        self.rf_phase_end.grid(row=8,column=5,sticky='w')
        self.freq_text.grid(row=9,column=2,sticky='e')
        self.act_freq.grid(row=9,column=3,columnspan=3,sticky='w')
        self.other_freq.grid(row=10,column=3,sticky='w')
        self.other_freq_entry.grid(row=10,column=4,columnspan=2,sticky='w')
        # Options for if grid of emitters is selected
        grid_rad_text.grid(row=13,column=0)
        self.grid_rad_entry.grid(row=13,column=1)
        grid_space_text.grid(row=14,column=0)
        self.grid_space_entry.grid(row=14,column=1,sticky='w')
        # Options for if custom pattern of emitters is selected
        num_emit_text.grid(row=13,column=0)
        self.num_emit_entry.grid(row=13,column=1)
        emit_loc_text.grid(row=14,column=0,columnspan=2,sticky='w')
        self.emit_loc_entry.grid(row=15,column=0,columnspan=2,rowspan=2)
        # Inputs/Options for Saving File
        slct_folder_btn.grid(row=14,column=2,columnspan=2,sticky='e')
        run_save_btn.grid(row=14,column=4,sticky='e')
        filename_text.grid(row=12,column=2,columnspan=3,sticky='nw')
        self.filename_entry.grid(row=13,column=2,columnspan=4,sticky='nw')
        self.progress_bar.grid(row=15,column=2,columnspan=4,sticky='new')
        
        self.window.mainloop()

    def pulsepower_rf_command(self):
        """
        This function is used to restructure the layout of the GUI based on whether the RF Cycle
        or Pulse Power option is selected. Each option requires different inputs, and this function is used to
        edit which inputs appear in the GUI based on which option is selected.

        """
        if self.pulsepower_rf_var.get() == 1: # RF Cycle option selected
            self.charge_text['text'] = 'Total Charge [nC]:'
            self.freq_text.grid(row=9,column=2,sticky='e')
            self.act_freq.grid(row=9,column=3,columnspan=3,sticky='w')
            self.other_freq.grid(row=10,column=3,sticky='w')
            self.other_freq_entry.grid(row=10,column=4,columnspan=2,sticky='w')
            self.phi_text.grid(row=2,column=2,columnspan=2,sticky='e')
            self.phi_entry.grid(row=2,column=4,columnspan=2,sticky='w')
            self.per_pulse.grid(row=7,column=0,sticky='w',columnspan=2)
            self.per_PL.grid(row=8,column=0,sticky='w',columnspan=2)
            self.local_field_text.grid(row=6,column=2,columnspan=2,sticky='e')
            self.local_field_entry.grid(row=6,column=4,columnspan=2,sticky='w')
            self.phase_shift_text.grid(row=7,column=2,columnspan=2,sticky='e')
            self.phase_shift_entry.grid(row=7,column=4,columnspan=2,sticky='w')
            self.rf_phase_text.grid(row=8,column=2,sticky='e')
            self.rf_phase_start.grid(row=8,column=3,sticky='e')
            self.rf_phase_dash.grid(row=8,column=4)
            self.rf_phase_end.grid(row=8,column=5,sticky='w')
        elif self.pulsepower_rf_var.get() == 2: # Pulse Power option selected
            self.charge_text['text'] = 'Current [amps]:'
            self.freq_text.grid_remove()
            self.act_freq.grid_remove()
            self.other_freq.grid_remove()
            self.other_freq_entry.grid_remove()
            self.phi_text.grid_remove()
            self.phi_entry.grid_remove()
            self.per_pulse.grid_remove()
            self.per_PL.grid_remove()
            self.local_field_text.grid_remove()
            self.local_field_entry.grid_remove()
            self.phase_shift_text.grid_remove()
            self.phase_shift_entry.grid_remove()
            self.rf_phase_text.grid_remove()
            self.rf_phase_start.grid_remove()
            self.rf_phase_dash.grid_remove()
            self.rf_phase_end.grid_remove()
        
    def change_emitter(self):
        """
        This function is used to restructure the layout of the GUI based on whether one emitter, 
        grid of emitters, or custom pattern of emitters is selected. The grid and custom options
        require additional inputs of the number of emitters and their locations for the custom pattern
        option. This function makes the necessary inputs visible based on which emitter option is selected.

        """
        if self.emitter_var.get() == 1: # One emitter selected
            try:
                self.if_grid_frame.grid_remove()
            except:
                pass
            try:
                self.if_custom_frame.grid_remove()
            except:
                pass
            try:
                self.charge_grid_custom_frame.grid_remove()
            except:
                pass
        elif self.emitter_var.get() == 2: # Grid of emitters selected
            self.charge_grid_custom_frame.grid(row=10,column=0,columnspan=2,rowspan=3,sticky='ew')
            self.if_grid_frame.grid(row=13,column=0,columnspan=2,rowspan=2,sticky='new')
            try:
                self.if_custom_frame.grid_remove()
            except:
                pass
        elif self.emitter_var.get() == 3: # Custom pattern of emitters selected
            self.charge_grid_custom_frame.grid(row=10,column=0,columnspan=2,rowspan=3,sticky='ew')
            self.if_custom_frame.grid(row=13,column=0,rowspan=4,columnspan=2,sticky='ew')
            try:
                self.if_grid_frame.grid_remove()
            except:
                pass

    def slct_folder(self):
        """
        This function is used to allow the user to select the location in which the constructed 
        distribution file will be saved.
        
        """
        self.save_folder = filedialog.askdirectory()

    def run(self):
        """
        Using the values entered and options selected by the user, this function creates the distributions
        for position, momentum, time, total charge, species, and flags to be stored in the data file.
        
        """
        s = Style()
        s.theme_use('clam')
        s.configure("red.Horizontal.TProgressbar", foreground='red', background='red')
        self.progress_bar['style'] = "red.Horizontal.TProgressbar"
        self.progress_var.set(0)
        self.progress_bar.update()
        
        cath_rad = float(self.cath_rad_var.get())
        n_particles = round(float(self.n_particles_entry.get()))
        Ek = float(self.Ek_entry.get())
        
        if self.pulsepower_rf_var.get() == 1: # RF cycle selected
            if self.act_freq_var.get() == True: # default ACT frequency selected
                freq = 1316.127688539e6 # ACT frequency in Hz
            else: # user entered alternate frequency
                freq = float(self.other_freq_entry.get())*1e9 # converting user entered frequency from GHz to Hz

            phi = float(self.phi_entry.get()) # [eV]
            charge = float(self.charge_var.get()) # [nC]
            rf_phase_start = float(self.rf_phase_start.get()) # [deg]
            rf_phase_end = float(self.rf_phase_end.get()) # [deg]
            local_field = float(self.local_field_entry.get())*1e9 #convert from GV/m to V/m
            phase_shift = float(self.phase_shift_entry.get()) # [deg]

            self.progress_var.set(20)
            self.progress_bar.update()

            if self.charge_pulse_type_var.get() == 1: # charge per pulse length selected
                PL = float(self.PL_entry.get()) # [seconds]

            if self.emitter_var.get() == 1: # one emitter selected
                if self.charge_pulse_type_var.get() == 1: # charge per pulse length selected
                    total_pulses = round(freq*PL)
                    charge = charge/total_pulses
            elif self.emitter_var.get() == 2 or self.emitter_var.get() == 3: # grid of emitters or custom pattern selected
                if self.emitter_var.get() == 2: # grid of emitters selected
                    n_emitters = int(self.grid_rad_entry.get())**2
                else: # custom pattern of emitters selected
                    n_emitters = int(self.num_emit_entry.get())

                if self.charge_pulse_type_var.get() == 1: # charge per pulse length selected
                    total_pulses = round(freq*PL)
                    if self.charge_rad_type_var.get() == 1: # charge per cathode radius selected
                        area_ratio = n_emitters*(float(self.emit_rad_entry.get())/cath_rad)**2
                        charge = charge/(area_ratio + total_pulses)
                    elif self.charge_rad_type_var.get() == 2: # charge per emitter radius selected
                        charge = charge/total_pulses
                elif self.charge_pulse_type_var.get() == 2 and self.charge_rad_type_var.get() == 1: # charge per RF Cycle and charge per cathode radius selected
                    area_ratio = n_emitters*(float(self.emit_rad_entry.get())/cath_rad)**2
                    charge = charge/area_ratio

            self.progress_var.set(30)
            self.progress_bar.update()

            # calculating sigt
            rf_phase = np.arange(0,359+0.01,0.01)

            Elocal = local_field*np.sin(np.deg2rad(rf_phase))

            I_emit = np.zeros_like(Elocal)

            for i in range(len(Elocal)):
                if Elocal[i]>0:
                    I_emit[i] = mp.mpf(Elocal[i]**2)*(mp.exp(-6.53e9*(phi**1.5)/Elocal[i]))
                    if I_emit[i] < 1e-302:
                        I_emit[i] = 0.0

            I_emit = I_emit/max(I_emit)
            
            def gaussian_func(x,a,b,c):
                """
                This function calculates a Gaussian distribution based on the given coefficients and
                x values to be used by Scipy's curve_fit function
                
                Inputs:  x, a, b, c
                Returns:  y (Gaussian distribution as a function of x)
                """
                y = a*np.exp(-((x-b)/c)**2)
                return y

            [a,b,c],pcov = curve_fit(gaussian_func,rf_phase,I_emit)
            sig_phi = c/(2**0.5)
            T = 1/freq
            sigt = T*sig_phi/360
            time = np.random.normal(0,(sigt*1e9)**0.5,n_particles) # creating time distribution [ns]
            t_shift = (T*phase_shift/360)*1e9 # [ns]
            time[0] = time[0] + t_shift # introducing time shift to time distribution
            # end calculating sigt


            
        elif self.pulsepower_rf_var.get() == 2: # Pulse Power selected
            current = float(self.charge_var.get()) # [amps]
            PL = float(self.PL_entry.get()) # [seconds]
            charge = current*PL*1e9 # [nC]
            
            self.progress_var.set(20)
            self.progress_bar.update()

            if self.emitter_var.get() == 1: # one emitter selected
                pass
            elif self.emitter_var.get() == 2 or self.emitter_var.get() == 3: # grid of emitters or custom pattern selected
                if self.emitter_var.get() == 2: # grid of emitters selected
                    n_emitters = int(self.grid_rad_entry.get())**2
                else: # custom pattern of emitters selected
                    n_emitters = int(self.num_emit_entry.get())
                if self.charge_rad_type_var.get() == 1: # charge per cathode radius selected
                    area_ratio = n_emitters*(float(self.emit_rad_entry.get())/cath_rad)**2
                    charge = charge/area_ratio
                    
            self.progress_var.set(30)
            self.progress_bar.update()
        
            time = np.random.uniform(-PL*1e9/2,PL*1e9/2,n_particles) # creating time distribution [ns]
            
        #making momentum distribution, see fuction 'make_momentums' for more details on momentum distributions
        px,py,pz = self.make_momentums(Ek,n_particles)
        Px_list = list(px)
        Py_list = list(py)
        Pz_list = list(pz)

        self.progress_var.set(65)
        self.progress_bar.update()

        #creating position distributions based on cathode radius
        x = np.zeros(n_particles)
        y = np.zeros(n_particles)
        z = np.zeros(n_particles)
        for i in range(n_particles):
            store_position = False 
            while store_position == False: # checking that the x,y position obeys the bounds of the cathode radius
                x_i = np.random.uniform(-cath_rad,cath_rad)
                y_i = np.random.uniform(-cath_rad,cath_rad)
                if x_i**2 + y_i**2 <= cath_rad**2:
                    store_position = True
            x[i] = x_i
            y[i] = y_i

        self.progress_var.set(75)
        self.progress_bar.update()

        #modifying lists if there are multiple emitters, look at function 'multiple_emitters' for more explanation
        if self.emitter_var.get() == 2 or self.emitter_var.get() == 3: # grid of emitters or custom pattern selected
            x,y,z,Px_array,Py_array,Pz_array,time = self.multiple_emitters(x,y,z,Px_list,Py_list,Pz_list,time)
        else:
            Px_array = np.array(Px_list) # change lists to arrays for uniformity because 'multiple_emitters' function returns values as arrays
            Py_array = np.array(Py_list) 
            Pz_array = np.array(Pz_list)

        self.progress_var.set(95)
        self.progress_bar.update()

        z = np.zeros_like(x) # creating uniform z distribution where all z=0
        total_charge = np.full((len(z)),(float(-charge/n_particles))) # creating uniform total charge distribution
        species = np.ones_like(z) # all species = 1
        flags = -1*np.ones_like(z) # all flags = -1 

        self.save_file(x,y,z,Px_array,Py_array,Pz_array,time,total_charge,species,flags)
            
    def make_momentums(self,ek,n_particles):
        """
        This function creates an isotropic momentum distribution based on the input value of initial energy.
        The constructed momentum distributions are designed to mimic those created by ASTRA for a uniform 
        energy distribution and isotropic momentum distribution. The generated momentum distributions 
        are used to calculate the kinetic energy distribution. The momentums are accepted and stored only if 
        the calculated kinetic energy distribution passes a test of uniformity with a significance level of 0.01.
        The statistical test of uniformity is done with Scipy's Stats kstest function.
        
        Inputs:  self, initial energy [eV], number of particles
        Outputs:  Px, Py, Pz (3D momentum)
        """
        accept_momentum = False 
        alpha = 0.01 # set alpha value to later compare statistical p-value to
        while accept_momentum == False: #create a condition that momentums must pass to be stored 
            progress_var = self.progress_var.get() #updating progress bar
            self.progress_var.set(progress_var+1)
            self.progress_bar.update()
            
            maximum_energy = ((ek+511e3)**2 - 511e3**2)**0.5
            unit_vector = np.random.normal(size=(n_particles, 3)) 
            unit_vector /= np.linalg.norm(unit_vector, axis=1)[:, np.newaxis] # creating "unit vector" distribution for 3D momentum where the total vector has a magnitude of 1
            px = unit_vector[:,0] * maximum_energy * np.random.uniform(1,1.02,size=n_particles) # multiplying "unit vectors" by maximum energy with an adjustment for error approximated to mirror that of ASTRA
            py = unit_vector[:,1] * maximum_energy * np.random.uniform(1,1.002,size=n_particles)
            pz = abs(unit_vector[:,2]) * maximum_energy
            for i in range(len(pz)): 
                pz[i] = pz[i] - maximum_energy
            # the following lines adjust the distributions to mimic those created by ASTRA
            px[0] = 0
            py[0] = 0
            pz[0] = maximum_energy
            px[1:7] = 0
            py[1:7] = 0
            pz[1:7] = -5.6843e-14
            momentum_vector = (px**2+py**2+pz**2)**0.5
            kinetic_energy = (momentum_vector**2 + 511e3**2)**0.5 - 511e3
            # testing kinetic energy distribution for uniformity
            ktest_stat, pvalue = stats.kstest(kinetic_energy,stats.uniform(loc=0,scale=ek*2).cdf)
            if pvalue < alpha: # comparison of statistical p-value to significance value of alpha=0.01
                accept_momentum = True
        return px,py,pz
            
    def multiple_emitters(self,orig_x,orig_y,z,Px_list,Py_list,Pz_list,time):
        """
        This function is used to modify the already created position (x,y,z), momentum (Px,Py,Pz), and time
        distributions to be accurate for multiple emitters if the grid of emitters or custom pattern of emitters
        options are selected.
        
        Inputs:  original distributions (for single emitter) of x,y,z,Px,Py,Pz, and time
        Outputs:  modifed distributions (for multiple emitters) of x,y,z,Px,Py,Pz, and time
        """
        final_x = np.array([])
        final_y = np.array([])
        if self.emitter_var.get() == 2: # if grid of emitters is selected
            n_emitters = int(self.grid_rad_entry.get())**2 # total number of emitters calculated from number of emitters per side of square grid
            grid_rad = int(self.grid_rad_entry.get()) # number of emitters per side of square grid
            grid_space = float(self.grid_space_entry.get())/1e3 # convert mm spacing to m

            # the following loop adjusts the x,y positions to be accurate for the grid of emitters
            for i in range(grid_rad): 
                for j in range(grid_rad):
                    adjust_x = grid_space*(grid_rad-(2*i)-1)/2
                    adjust_y = grid_space*(grid_rad-(2*j)-1)/2
                    new_x = orig_x + adjust_x
                    new_y = orig_y + adjust_y
                    final_x = np.concatenate((final_x,new_x))
                    final_y = np.concatenate((final_y,new_y))
                    
        elif self.emitter_var.get() == 3: # custom pattern of emitters selected
            n_emitters = int(self.num_emit_entry.get())
            locs = self.emit_loc_entry.get("1.0", tk.END)
            sep_locs = locs.split(';') # splitting list of locations into x,y pairs
            x_locs = []
            y_locs = []
            for i in range(len(sep_locs)):
                sep_locs[i] = sep_locs[i].split(",")
                x_locs.append(float(sep_locs[i][0])*1e-3) # storing the custom pattern locations
                y_locs.append(float(sep_locs[i][1])*1e-3) # and converting from mm to m

            for i in range(len(x_locs)):
                new_x = orig_x + x_locs[i] # adjusting the existing position distributions
                new_y = orig_y + y_locs[i]
                final_x = np.concatenate((final_x,new_x))
                final_y = np.concatenate((final_y,new_y))
        
        final_z = np.zeros_like(final_x)
        Px_array = np.array(Px_list*n_emitters) # replicating the momentum distributions for each emitter location
        Py_array = np.array(Py_list*n_emitters)
        Pz_array = np.array(Pz_list*n_emitters)
        final_time = np.array(list(time)*n_emitters) # replicating the time distribution for each emitter location
        
        return final_x,final_y,final_z,Px_array,Py_array,Pz_array,final_time
    
  
    def save_file(self,x,y,z,Px,Py,Pz,time,total_charge,species,flags):
        """
        This function is used to save the generated distributions to a .ini file named by the user in the location
        they selected. The file is formatted into columns: x,y,z,Px,Py,Pz,time,total charge, species, and flags.
        Species is set to 1 and flags is set to -1 for all particles. The numeric formatting for x,y,z,Px,Py,Pz,
        and time is in exponent notation of a floating point number. Species and flags are formatted as integers.
        When the file has saved, the progress bar at the bottom of the GUI will become green.
        """
        full_save_filename = self.save_folder + "/" + self.filename_entry.get() + '.ini'
        save_format = ['%+.12e','%+.12e','%+.12e','%+.12e','%+.12e','%+.12e','%+.12e','%+.12e','%3i','%3i']
        np.savetxt(full_save_filename, np.column_stack([x,y,z,Px,Py,Pz,time,total_charge,species,flags]),fmt=save_format)
        s = Style()
        s.theme_use('clam')
        s.configure("green.Horizontal.TProgressbar", foreground='green', background='green')
        self.progress_bar['style'] = "green.Horizontal.TProgressbar"
        self.progress_bar.stop()
        self.progress_var.set(100)
        self.progress_bar.update()



In [5]:
FEgen_v1()